In [1]:
import torch

d:\DS\envs\gan-project\lib\site-packages\torch\_subclasses\functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
# input noise --> concatenate one-hot label
# fully connected layer
# reshape to make matrix
# conv 2d to make channels
# convtranspose 2d to upsample to 28x28 dimensions of MNIST


In [ ]:
import torch.nn as nn

class MaskGenerator(nn.Module):
    def __init__(self, in_channels=1, latent_dim=128):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=4, stride=2, padding=1),  # 96x96 → 48x48
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),           # 48x48 → 24x24
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),          # 24x24 → 12x12
            nn.ReLU()
        )

        self.flatten = nn.Flatten()
        self.fc_mu = nn.Linear(128 * 12 * 12, latent_dim)
        self.fc_logvar = nn.Linear(128 * 12 * 12, latent_dim)

    def forward(self, x):
        x = self.encoder(x)                    # shape: (B, 128, 12, 12)
        x = self.flatten(x)                    # shape: (B, 128*12*12)
        mu = self.fc_mu(x)                     # shape: (B, latent_dim)
        logvar = self.fc_logvar(x)             # shape: (B, latent_dim)
        return mu, logvar


In [3]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, out_channels=1, latent_dim=128):
        super().__init__()

        self.fc = nn.Linear(latent_dim, 128 * 12 * 12)

        self.decoder = nn.Sequential(
            nn.Unflatten(1, (128, 12, 12)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 12 → 24
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),   # 24 → 48
            nn.ReLU(),
            nn.ConvTranspose2d(32, out_channels, kernel_size=4, stride=2, padding=1)  # 48 → 96
        )

    def forward(self, x):
        x = self.fc(x)         # (B, latent_dim) → (B, 128*12*12)
        x = self.decoder(x)    # (B, 128*12*12) → (B, out_channels, 96, 96)
        return x


The whole idea of this project will be to imporve CNN classifier performance by expanding dataset

When taking slices of tumor images, keep track of depth, create distrivution
To create slices of healthy images, take samples from same distribution